# Generate pyspark Notebook

This Notebook generates a notebook (see last cell) which can be run in pyspark. This Notebook itself does not need Spark.

In [1]:
import nbformat as nbf
import nbformat.v4 as nbf4

In [2]:
nb_out = nbf4.new_notebook()
cells_out = []

First we create the workflow, by reading in Notebooks and defining the flow.

In [3]:
with open("../Rig/blocks/load_csv.ipynb", 'r') as f:
    input_block = nbf.read(f, 4)
    
with open("../Rig/blocks/sample.ipynb", 'r') as f:
    block1 = nbf.read(f, 4)
    
workflow = [input_block, block1]

In [4]:
##Add the preamble
with open("../Rig/blocks/preamble.ipynb", 'r') as f:
    preamble = nbf.read(f, 4)
cells_out.extend(preamble['cells'])

Now we are going to loop of over the workflow and create the cells for the notebook. It might need input from the user.

In [5]:
last_output = None
for block in workflow:
    
    for cell in block['cells']:
        metadata = cell.get('metadata', {})
        name = metadata.get("blockname", "Unknown block")
        if metadata.get('type', '') == 'block' :
            
            namecell = "## %s" % name
            cells_out.append(nbf4.new_markdown_cell(namecell))
            
            #create the input
            code_input = '%s = %s \n' % (metadata.get('input', 'input'), last_output)
            cells_out.append(nbf4.new_code_cell(code_input))

            #Create the arguments
            code_arguments = ""
            arguments = metadata.get('arguments', [])
            for argument in arguments:
                print "give input for parameter %s in block %s" % (argument, name)
                val = raw_input()
                code_arguments += ("%s = %s \n" % (argument, val))
            if len(arguments) > 0:
                cells_out.append(nbf4.new_code_cell(code_arguments))
            code_arguments

            #append the actual cell
            cells_out.append(cell)
            last_output = metadata.get('output', None)

give input for parameter path in block load csv
'/home/jovyan/work/Crimes_-_2001_to_present_2015-12.csv'
give input for parameter sample_size in block sample
100


In [6]:
nb_out['cells'] = cells_out

In [7]:
with open('../blocks/generated_test_spark.ipynb', 'w') as f:
    nbf.write(nb_out, f, version=4)